In [2]:
import requests

url = "http://127.0.0.1:8800"

In [7]:
# get environment coordinates
response = requests.get(f"{url}/scene_objects")
print(response.json())

{'count': 188, 'objects': [{'name': 'bottom_left_group_1_door_handle_main', 'position': {'x': 0.5850447070866696, 'y': -4.0999987906572475, 'z': 0.2}, 'quaternion': {'w': 0.7071052811843601, 'x': 0.0, 'y': 0.0, 'z': 0.707108281185553}, 'yaw': 1.5708005694372709}, {'name': 'bottom_left_group_1_door_main', 'position': {'x': 0.5850447070866696, 'y': -4.0999987906572475, 'z': 0.2}, 'quaternion': {'w': 0.7071052811843601, 'x': 0.0, 'y': 0.0, 'z': 0.707108281185553}, 'yaw': 1.5708005694372709}, {'name': 'bottom_left_group_1_inner_box', 'position': {'x': 0.30004470708923453, 'y': -4.099999999810324, 'z': 0.2}, 'quaternion': {'w': 0.7071052811843601, 'x': 0.0, 'y': 0.0, 'z': 0.707108281185553}, 'yaw': 1.5708005694372709}, {'name': 'bottom_left_group_1_main', 'position': {'x': 0.3, 'y': -4.1, 'z': 0.2}, 'quaternion': {'w': 0.7071052811843601, 'x': 0.0, 'y': 0.0, 'z': 0.707108281185553}, 'yaw': 1.5708005694372709}, {'name': 'bottom_left_group_2_door_handle_main', 'position': {'x': 0.585044707086

In [8]:
response = requests.get(f"{url}/robot_pose")
print(response.json())

{'current': {'x': -2.0027000773332177e-07, 'y': -7.843899158122813e-08, 'theta': 2.139812325384922e-05}, 'target': {'x': 0.0, 'y': 0.0, 'theta': 0.0}, 'velocity': {'x': 1.4541768739357017e-10, 'y': 5.93566053022243e-11, 'theta': -2.5170843500559154e-07}, 'error': {'x': 2.0027000773332177e-07, 'y': 7.843899158122813e-08, 'theta': -2.139812325384922e-05}, 'end_effector': {'position': {'x': 0.6400050883173486, 'y': -3.4500100211880116, 'z': 1.556972088914508}, 'quaternion': {'w': 0.04829760474117823, 'x': 0.8856750308329083, 'y': -0.45198663210233747, 'z': -0.09463173641108773}, 'yaw': -0.9545647591574836}}


In [10]:
payload = {
    "action": {
        "type": "run_code",
        "payload": {
            "code": """set_target_position(0, 0, PI), set_target_position(-0.5, 0, PI)"""
        },
    }
}
response = requests.post(f"{url}/send_action", json=payload)